In [1]:
from rouge import Rouge, FilesRouge
import pandas as pd
import numpy as np
import datetime
import glob
import os
rouge = Rouge()

In [2]:
# get max csv
# get max file
list_of_files = glob.glob('Z:/final_data/scored_data/notes_scored*.csv') # * means all if need specific format then *.csv
ckpt_text = max(list_of_files, key=os.path.getctime)
print(ckpt_text)

Z:/final_data/scored_data\notes_scored_2018-10-02_21_38_28.csv


In [3]:
# read in actual notes
notes = pd.read_csv(ckpt_text,index_col=0,keep_default_na=False)
notes.columns = ['findings','impressions','ref_impressions']

In [4]:
notes.head()

,scored_summaries
18,Increase in diffuse pulmonary opacities which ...
19,Diffuse nonspecific interstitial disease of un...
20,Right chest tube in place with no significant ...
23,Worsening pulmonary opacities still suggestive...
24,Diffuse nonspecific interstitial disease of un...


In [5]:
# join notes to the test set
test_dta = pd.read_csv("Z:/final_data/cohorts_merged_test.csv",keep_default_na=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,69,325) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# join in the columns 
# test_dta.join(notes)
scored_df = pd.merge(test_dta, notes, left_index=True, right_index=True,how="left")

In [12]:
scored_df.loc[23:25]

,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death,encounter_id,enc_eio,visit_status_1,visit_status_2,...,findings,impressions,elecsig,patient_raw,rawtext,encounter_id_diagnosed,date_diagnosed,ild_diag_prior_to_visit,ild_status,scored_summaries
23,114,66,F,White,Not Hispanic or Latino,,9495002,I,No Show,Completed,...,LUNGS AND PLEURA: Significant progression of g...,Worsening pulmonary opacities still suggestive...,**DATE<[**2019-06-12**]> 4:21 PM **NAME<LLL MM...,9495002_653735,CT CHEST WO **DATE<[**2019-06-12**]> 3:49 PM\...,4645531,2013-08-22,1,1,Worsening pulmonary opacities still suggestive...
24,114,66,F,White,Not Hispanic or Latino,,9495002,I,No Show,Completed,...,"Diffuse nonspecific interstitial opacity, not ...",Diffuse nonspecific interstitial disease of un...,**DATE<[**2019-07-28**]> 9:06 AM **NAME<PPP QQ...,9495002_662175,"XR PORT CHEST 1V, **DATE<[**2019-07-27**]> 10:...",4645531,2013-08-22,1,1,Diffuse nonspecific interstitial disease of un...
25,114,66,F,White,Not Hispanic or Latino,,9495002,I,No Show,Completed,...,,,,,,4645531,2013-08-22,1,1,NaN


In [8]:
test_tmp = scored_df.loc[23:24]

#### define functions

In [9]:
def avg_rouge_score(df,column_string):
    avg_score_list = []
    avg_score_cols = []
    for col in df.columns:
        if column_string in col:
            print("[info] string found", col)
            avg_score_cols.append(col)
            avgscore = np.nanmean(df[col])
            avg_score_list.append(avgscore)
    tmpdf = pd.DataFrame({
        'avg_score_cols':avg_score_cols,
        'score':avg_score_list
    })
    return(tmpdf)

In [10]:
def score_data(df, predicted, actuals):
    printcounter = 0
    for index,row in df.iterrows():
#         print(printcounter)
        printcounter += 1
        if(printcounter % 1000)==0:
            print("[info] 1000 iterations")
#         print("[info] index #: ",index)
#         print(row[predicted])
#         print(row[actuals])
        if str(row[predicted])!="nan":
#             print("[info] this was not a null row. Logging index")
            indexlist.append(index)
            rougescore = rouge.get_scores(row[predicted],row[actuals])
            tmpdf = pd.DataFrame(rougescore)
#             print("[info] now appending to list rouge values")
            for row in tmpdf:
                scorelist.append(tmpdf[row].apply(pd.Series).values[0])
        else:
            continue

#### now run through

In [11]:
# get np array now of rouge scores
scorelist = []
indexlist = []
score_data(scored_df,'impressions','ref_impressions')
print(scorelist)
print(indexlist)

[array([0.12068965, 0.175     , 0.09210526]), array([0.04580152, 0.06382979, 0.03571429]), array([0.10265301, 0.175     , 0.09210526]), array([0.3076923 , 0.36363636, 0.26666667]), array([0.16666666, 0.2       , 0.14285714]), array([0.29409265, 0.36363636, 0.26666667])]
[23, 24]


In [12]:
rouge.get_scores(test_tmp.impressions,)

[{'rouge-1': {'f': 0.12068965065398352, 'p': 0.175, 'r': 0.09210526315789473},
  'rouge-2': {'f': 0.04580152211642725,
   'p': 0.06382978723404255,
   'r': 0.03571428571428571},
  'rouge-l': {'f': 0.10265300928877165, 'p': 0.175, 'r': 0.09210526315789473}},
 {'rouge-1': {'f': 0.3076923028106509,
   'p': 0.36363636363636365,
   'r': 0.26666666666666666},
  'rouge-2': {'f': 0.1666666618055557, 'p': 0.2, 'r': 0.14285714285714285},
  'rouge-l': {'f': 0.29409264768364346,
   'p': 0.36363636363636365,
   'r': 0.26666666666666666}}]

In [13]:
# assign
rouged_df = pd.DataFrame(np.array(scorelist).reshape(int(len(scorelist)/3),9),columns=['rouge-1-f','rouge-1-p','rouge-1-r','rouge-2-f','rouge-2-p','rouge-2-r','rouge-l-f','rouge-l-p','rouge-l-r'],index=indexlist)
rouged_df.head(n=10)


,rouge-1-f,rouge-1-p,rouge-1-r,rouge-2-f,rouge-2-p,rouge-2-r,rouge-l-f,rouge-2-p,rouge-2-r
23,0.120690,0.175000,0.092105,0.045802,0.06383,0.035714,0.102653,0.175000,0.092105
24,0.307692,0.363636,0.266667,0.166667,0.20000,0.142857,0.294093,0.363636,0.266667


In [17]:
scored_rouged_df = pd.merge(scored_df, rouged_df, left_index=True, right_index=True,how="left")

In [18]:
scored_rouged_df.head(n=20)

,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death,encounter_id,enc_eio,visit_status_1,visit_status_2,...,scored_summaries,rouge-1-f,rouge-1-p,rouge-1-r,rouge-2-f,rouge-2-p,rouge-2-r,rouge-l-f,rouge-2-p,rouge-2-r
0,49,79,M,White,Not Hispanic or Latino,,10440586,O,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49,79,M,White,Not Hispanic or Latino,,10440586,O,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49,79,M,White,Not Hispanic or Latino,,10440586,O,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49,79,M,White,Not Hispanic or Latino,,10440586,O,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49,79,M,White,Not Hispanic or Latino,,11420608,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,49,79,M,White,Not Hispanic or Latino,,11420608,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,49,79,M,White,Not Hispanic or Latino,,11420608,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,49,79,M,White,Not Hispanic or Latino,,11420608,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,49,79,M,White,Not Hispanic or Latino,,11455337,O,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,104,84,M,White,Not Hispanic or Latino,,9339110,O,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
scored_rouged_df.columns

Index(['patient_id', 'age_at_last_encounter', 'gender', 'race', 'ethnicity',
       'date_of_death', 'encounter_id', 'enc_eio', 'visit_status_1',
       'visit_status_2',
       ...
       'scored_summaries', 'rouge-1-f', 'rouge-1-p', 'rouge-1-r', 'rouge-2-f',
       'rouge-2-p', 'rouge-2-r', 'rouge-l-f', 'rouge-2-p', 'rouge-2-r'],
      dtype='object', length=352)

In [20]:
avg_score_df = avg_rouge_score(scored_rouged_df,"rouge")
print(avg_score_df)

[info] string found rouge-1-f
[info] string found rouge-1-p
[info] string found rouge-1-r
[info] string found rouge-2-f
[info] string found rouge-2-p
[info] string found rouge-2-r
[info] string found rouge-l-f
[info] string found rouge-2-p
[info] string found rouge-2-r
  avg_score_cols     score
0      rouge-1-f  0.274379
1      rouge-1-p  0.538104
2      rouge-1-r  0.202274
3      rouge-2-f  0.148304
4      rouge-2-p  0.413897
5      rouge-2-r  0.151741
6      rouge-l-f  0.207654
7      rouge-2-p  0.413897
8      rouge-2-r  0.151741


In [21]:
# filename = "/gpfs/data/ildproject-share/final_data/scored_data/avg_score_%s.csv" % (str(datetime.datetime.now()).split('.')[0].replace(' ','_').replace(':','_'))
filename = "Z:/final_data/scored_data/avg_score_%s.csv" % (str(datetime.datetime.now()).split('.')[0].replace(' ','_').replace(':','_'))

In [22]:
scored_rouged_df.to_csv("Z:/final_data/scored_data/scored_rouged_df.csv")

In [23]:
avg_score_df.to_csv(filename)